# В этом ноутбуке будут рассмотрены различные видны препроцессинга данных для моделей классификации

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import KDTree
import sys

sys.path.append('/Users/alexander.savelyev/PycharmProjects/Snow_recognition')
from preprocess import Preprocess

preprocess.py -- в этом файле написаны все методы препроцессинга, которые будут описываться в этой тетрадке 

In [2]:
preprocessor = Preprocess()

In [3]:
df = pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data.csv")

In [4]:
df.head(5)

,scene_id,x,y,z,intensity,ring,label
0,0,-11.355618,-4.206962,0.344085,0.0,23.0,1.0
1,0,-5.916535,-1.972164,0.283262,0.0,25.0,1.0
2,0,-7.410451,-2.113039,2.137792,0.0,31.0,1.0
3,0,-13.845870,-1.406652,0.406310,0.0,23.0,1.0
4,0,-8.326218,-0.346060,0.226469,0.0,22.0,1.0


### 1. Простой препроцессинг

Для каждой сцены будем в ней нормализовывать с помощью min_max в ней координаты x, y, z, intensity и ring

In [8]:
df_1 = preprocessor.simple_preprocess_normalize_all_columns(df)
df_1.to_csv("/Volumes/HP P800/itmo/Lidar data/data_simple_preprocess_normalize_all_columns.csv")
df_1 = pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data_simple_preprocess_normalize_all_columns.csv", index_col=0)

In [9]:
df_1

,scene_id,x,y,z,intensity,ring,label,x_normailized,y_normailized,z_normailized,ring_normailized,intensity_normailized
0,0,-11.355618,-4.206962,0.344085,0.0,23.0,1.0,0.334384,0.428199,0.304751,0.741912,0.000000
1,0,-5.916535,-1.972164,0.283262,0.0,25.0,1.0,0.425570,0.466143,0.298890,0.806426,0.000000
2,0,-7.410451,-2.113039,2.137792,0.0,31.0,1.0,0.400525,0.463751,0.477613,0.999968,0.000000
3,0,-13.845870,-1.406652,0.406310,0.0,23.0,1.0,0.292635,0.475745,0.310748,0.741912,0.000000
4,0,-8.326218,-0.346060,0.226469,0.0,22.0,1.0,0.385172,0.493753,0.293417,0.709655,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5587413,229,-21.061958,-19.518799,-1.736692,9.0,7.0,0.0,0.168736,0.168792,0.223123,0.225799,0.035856
5587414,229,-14.454390,-13.593932,-2.120711,17.0,4.0,0.0,0.279571,0.268292,0.184520,0.129028,0.067729
5587415,229,-16.462903,-16.933895,-2.049159,21.0,5.0,0.0,0.245881,0.212202,0.191713,0.161285,0.083665
5587416,229,-14.510767,-13.547505,-2.122062,17.0,4.0,0.0,0.278626,0.269071,0.184384,0.129028,0.067729


Проверим что координаты действительно нормализованы

In [10]:
df_1.describe()

,scene_id,x,y,z,intensity,ring,label,x_normailized,y_normailized,z_normailized,ring_normailized,intensity_normailized
count,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06,5.587418e+06
mean,1.129268e+02,6.404926e-01,7.356524e-02,-9.317976e-01,1.544064e+01,1.223832e+01,1.052280e-01,5.309799e-01,5.048388e-01,1.775312e-01,3.740787e-01,1.057433e-01
std,6.651930e+01,1.189973e+01,1.235524e+01,1.020410e+00,1.759814e+01,9.141331e+00,3.068470e-01,2.019528e-01,2.125203e-01,1.190406e-01,2.644248e-01,1.230880e-01
min,0.000000e+00,-3.149820e+01,-2.998987e+01,-5.708614e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,-7.758378e+00,-8.026161e+00,-1.692540e+00,4.000000e+00,5.000000e+00,0.000000e+00,3.869205e-01,3.634907e-01,8.975860e-02,1.612851e-01,2.564092e-02
50%,1.110000e+02,1.964681e+00,-1.041601e+00,-1.123072e+00,1.200000e+01,1.000000e+01,0.000000e+00,5.516589e-01,4.849513e-01,1.570859e-01,2.903132e-01,7.058796e-02
75%,1.700000e+02,8.635318e+00,8.710825e+00,-3.801832e-01,2.000000e+01,1.800000e+01,0.000000e+00,6.683471e-01,6.517050e-01,2.400947e-01,5.483694e-01,1.415081e-01
max,2.290000e+02,2.998755e+01,2.999027e+01,8.677580e+00,2.550000e+02,3.900000e+01,1.000000e+00,9.999833e-01,9.999833e-01,9.999246e-01,9.999744e-01,9.999961e-01


Как видим, x_normalized, y_normalized, z_normalized, ring_normailized и intensity_normailized находяться от 0 до 1

Будем использовать эти данные для обучения логистической регресии, потому что линейная модель может быть очень подвержена большой высокоими значениями фичей

### 2. Препроцессинг на основе KDTree

Для каждой сцены будем делать следующее:

1) Нормализовывать координаты x, y и z, а затем сохранять их в x_normalized, y_normalized и z_normalized.

2) Для всех точек сцены по кординатам x_normalized, y_normalized и z_normalized будем строить KDTree. KDTree -- структура для быстрого поиска ближайших соседей, будем использовать ее потому что полный перебор очень долго делается.

3) Затем для каждой точки сцены будем находить ее 7 ближайших соседей с помощью KDTree. 

4) Будем считать следующие характеристики на основе ее ближайших соседей:

    * dist_mean -- среднее расстояние от точки до всех ее соседей
    * dist_std -- стандартное отклонение расстояний от точки до всех ее соседей
    * dist_for_first -- расстоние до самой ближней точки (из найденных соседей)
    * dist_for_last -- расстоние до самой дальней точки точки (из найденных соседей)
    
    * ring_mean -- средний номер луча от точки до всех ее соседей
    * ring_std -- стандартное отклонение номера луча от точки до всех ее соседей
    * ring_for_first -- номера луча самой ближней точки (из найденных соседей)
    * ring_for_last -- номера луча самой дальней точки точки (из найденных соседей)
    
    * int_mean -- среднея интенсивность от точки до всех ее соседей
    * int_std -- стандартное отклонение интенсивности от точки до всех ее соседей
    * int_for_first -- интенсивность самой ближней точки (из найденных соседей)
    * int_for_last -- интенсивность самой дальней точки точки (из найденных соседей)
    

Такой фичи инжеринг обуслевден тем фактом, что проведя EDA мы сделали несколько выводов:

* Также дисперсия для снега меньше, чем у обычных точек, из этого можно сделать вывод, что снег скапливается в одном месте, а не "раскидан" по всей сцене.

* По этой информации можно сделать вывод, что снег в среднем находиться выше, чем другие точки, потому z_normailized больше для label = 1, чем z_normailized для label = 0.

* По этой информации можно сделать вывод, что снег в среднем намного тусклее, и эта тусклость находиться около 0.

Значит если точка находиться в окружении точек снега, то скорее всего она будет тоже снегом, эту гипотезу мы проверем в будущем

In [11]:
df_kdtree = preprocessor.kdtree_preprocess(df)
df_kdtree.to_csv("/Volumes/HP P800/itmo/Lidar data/data_kdtree_preprocess.csv")
df_kdtree = pd.read_csv("/Volumes/HP P800/itmo/Lidar data/data_kdtree_preprocess.csv")

In [12]:
df_kdtree

,Unnamed: 0,scene_id,x,y,z,intensity,ring,label,x_normailized,y_normailized,...,dist_for_first,dist_for_last,ring_mean,ring_std,ring_for_first,ring_for_last,int_mean,int_std,int_for_first,int_for_last
0,0,0,-11.355618,-4.206962,0.344085,0.0,23.0,1.0,0.334384,0.428199,...,0.063558,0.109770,16.666667,7.366591,25.0,10.0,20.333333,22.642144,0.0,48.0
1,1,0,-5.916535,-1.972164,0.283262,0.0,25.0,1.0,0.425570,0.466143,...,0.049237,0.076238,18.500000,4.593474,22.0,25.0,0.833333,1.329160,0.0,0.0
2,2,0,-7.410451,-2.113039,2.137792,0.0,31.0,1.0,0.400525,0.463751,...,0.160898,0.173386,27.833333,1.602082,27.0,31.0,0.000000,0.000000,0.0,0.0
3,3,0,-13.845870,-1.406652,0.406310,0.0,23.0,1.0,0.292635,0.475745,...,0.063558,0.103703,24.000000,1.673320,23.0,26.0,0.000000,0.000000,0.0,0.0
4,4,0,-8.326218,-0.346060,0.226469,0.0,22.0,1.0,0.385172,0.493753,...,0.041669,0.056021,22.333333,3.076795,25.0,25.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5587413,5587413,229,-21.061958,-19.518799,-1.736692,9.0,7.0,0.0,0.168736,0.168792,...,0.015266,0.032256,6.166667,0.408248,6.0,7.0,12.833333,2.316607,12.0,10.0
5587414,5587414,229,-14.454390,-13.593932,-2.120711,17.0,4.0,0.0,0.279571,0.268292,...,0.001207,0.006411,4.000000,0.000000,4.0,4.0,15.666667,1.032796,15.0,17.0
5587415,5587415,229,-16.462903,-16.933895,-2.049159,21.0,5.0,0.0,0.245881,0.212202,...,0.001430,0.007129,5.000000,0.000000,5.0,5.0,21.000000,0.000000,21.0,21.0
5587416,5587416,229,-14.510767,-13.547505,-2.122062,17.0,4.0,0.0,0.278626,0.269071,...,0.001233,0.007502,4.000000,0.000000,4.0,4.0,15.666667,1.032796,15.0,17.0
